In [ ]:
from selenium.webdriver.common.by import By
from _nytimes import Puzzle

from langchain.output_parsers import PydanticOutputParser
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

API_KEY=""

In [ ]:
browser = Puzzle("games/connections")
words = [ card.text for card in browser.find_elements(By.XPATH, "//label[contains(@for, 'inner-card')]") ]

In [ ]:
class Group(BaseModel):
    words:List[str] = Field(..., description="Group of four words belonging to the same category.", max_length=4)
    description:str = Field(..., description="Category's name.")

class Groups(BaseModel):
    guess:List[Group] = Field(..., description="Your guesses.")

class Player:
    def __init__(self):
        self.llm = GoogleGenerativeAI(model="gemini-2.0-flash", api_key=API_KEY, temperature=0)

        self.parser = PydanticOutputParser(pydantic_object=Groups)
        self.fmt = {"partial_variables":{"FORMAT":self.parser.get_format_instructions()}}

    def guess(self, words:list[str]) -> str:
        prompt = PromptTemplate(template="""
            You are playing the New York Times' game called 'connections'.
            Your goal is group the following words into groups of four where they must belong to the same category.
            {WORDS}
            The category can be anything. From the word's pronunciation to their meanings.
            {FORMAT}
            """, input_variables=["WORDS"], **self.fmt)
        return (prompt | self.llm | self.parser).invoke({"WORDS": words}).guess

In [ ]:
resp = Player().guess(words)

In [ ]:
browser.end()